# oneM2M - Groups

This notebook demonstrates how oneM2M resources can be organized into groups. &lt;Group> resources offer a so-called *fan-out point* that can be the target to address all resources of a group.

- Create a second &lt;Container resource
- Create a &lt;Group> resource with &lt;Container> resources
- Add a &lt;ContentInstance> to the &lt;Group>'s containers

## Intitialization
The section does import necessary modules and configurations.

In [ ]:
from init import *

## Create a second &lt;Container> Resource

First, we create a second &lt;Container> resource.


In [ ]:
CREATE (                                           # CREATE request
    url + '/' + ae(),
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=3'   # Content is JSON, and represents an <Container> resource
    },
    
    # Request Body
    '''
    {
      "m2m:cnt": {
        "rn":"secondContainer"
      }
    }
    '''
)

## Create a &lt;Group> Resource

Here, we create a &lt;Group> resource with the two &lt;Container> resources as components.

- **mid** : Member IDs, this is a list of resource IDs of resources that will belong to the &lt;Group>. 
- **mnm** : The maximum number of member of a &lt;Group>.

Note, that a &lt;Group> may contain resources of mixed type. Requests to the &lt;Group>'s fan-out point in general only affect fitting resources.


In [ ]:
CREATE (                                           # CREATE request
    url + '/' + ae(),
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=9'   # Content is JSON, and represents an <Group> resource
    },
    
    # Request Body
    '''
    {
      "m2m:grp": {
        "rn": "myGroup",
        "mid": [ "cse-in/%s/myContainer", "cse-in/%s/secondContainer" ],
        "mnm": 10
      }
    }
    ''' % (ae(), ae())
)

The response introduces some new attributes:

- **mt** : Member type. This is the resource type of the member resources. The member type is *mixed" in case the member resourtces are not all of the same type.
- **cnm** : The current number of member of the &lt;Group> resource.
- **mtv** : Member types validated. This read-only attribute indicates whether the hosting CSE validated the member resources.
- **csy** : The consistency strategy determines how the CSE deals with the &lt;Group> resource if the member type validation fails.
- **fopt** : The fan-out point is a virtual resource created by the CSE. It acts as a target for requests to the members of a &lt;Group> resource.


## Add a &lt;ContentInstance> Resource to all &lt;Group> Resources
The following request creates multiple &lt;ContentInstances> resources, one under each member of the &lt;Group> resource.

In [ ]:
CREATE (                                            # CREATE request
    url + '/' + ae() + '/myGroup/fopt',
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=4'   # Content is JSON, and represents an <ContentInstance> resource
    },
    
    # Request Body
    '''
    {
      "m2m:cin": {
        "cnf": "text/plain:0",
        "con": "Hello, World!"
      }
    }
    '''
)

The request returns all the created &lt;ContentInstnces>.

- **agr** : This response is an Agregated Response, since the call results in multiple resources.
- **rsp** : The list of responses.
